In [1]:
#import dependencies
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
import sqlite3
Base = declarative_base()

In [2]:
# Load CSV Files

# sources: https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results/data#
# Data starts in 1896
events_file = 'Resources/athlete_events.csv'
df_events_orig = pd.read_csv(events_file)

# source: World Bank (https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=#) 
# Data starts in 1960
pop_gdp_file = 'Resources/population_gdp.csv'
df_pop_gdp_orig = pd.read_csv(pop_gdp_file)

<h3> Clean Worldbank data

In [3]:
# step 1 - rename columns
df_pop_gdp = df_pop_gdp_orig
df_pop_gdp = df_pop_gdp.drop(columns = ['Series Code'], axis =1)
df_pop_gdp = df_pop_gdp.rename(
    columns={
        "Series Name": "Series", 
        "Country Name": "Country",
        "Country Code": "NOC"})

df_pop_gdp.columns = df_pop_gdp.columns.str.split(' ').str[0].tolist()

keep_columns = ['Series', 'Country', 'NOC']
event_years = df_events_orig.Year.unique().astype(str)

for i in event_years:
    keep_columns.append(i)

df_pop_gdp = df_pop_gdp[df_pop_gdp.columns.intersection(keep_columns)]

In [4]:
# step 2 - filter by GDP and Population series_names
df_pop_gdp_1 = df_pop_gdp.loc[(df_pop_gdp["Series"] == "GDP (current US$)") | (df_pop_gdp["Series"] == "Population, total")]
df_pop_gdp_2 = df_pop_gdp_1.melt(id_vars=['Series', 'Country', 'NOC'])
df_pop_gdp_2 = df_pop_gdp_2.rename(columns={'variable':'Year'})

In [5]:
# step 3 - Reformat the table
df_pop_gdp_final = df_pop_gdp_2.pivot_table(index=['Year','NOC','Country'], values = 'value', columns='Series', aggfunc='sum')
df_pop_gdp_final = df_pop_gdp_final.reset_index()
df_pop_gdp_final = df_pop_gdp_final.rename(columns = {'GDP (current US$)': 'GDP', 'Population, total': 'Population'})
df_pop_gdp_final['Population'] = np.where(df_pop_gdp_final['Population'] == '..', 0, df_pop_gdp_final['Population'])
df_pop_gdp_final['GDP'] = np.where(df_pop_gdp_final['GDP'] == '..', 0, df_pop_gdp_final['GDP'])
df_pop_gdp_final = df_pop_gdp_final.astype({'GDP': 'float64', 'Year': 'int64', 'Population': 'int64'})
df_pop_gdp_final['GDP_per_capita'] = df_pop_gdp_final['GDP'] / df_pop_gdp_final['Population']

<h3> Clean Olympic Events

In [6]:
# step 1 - define df_events dataframe
df_events = df_events_orig

In [7]:
# step 2 - clean NOC (in df_events) based on worldbank NOC to match each other

# extract WorldBank's NOC (Country code) and Country (name)
df_worldbanknames = pd.DataFrame(df_pop_gdp_final.groupby(['NOC','Country']).size().reset_index())
df_worldbanknames = df_worldbanknames.drop(columns=[0])

# create a column with stripped country_name and sanitize
df_worldbanknames['Wclean'] = df_worldbanknames ['Country'].str.replace(" ", "").str.lower()

# create a column with stripped Team name in Olympic Data and sanitize
df_events['Oclean'] = df_events ['Team'].str.replace(" ", "").str.lower()

# Unique transformations of countries that are in both data bases, but didn't match up.
df_events['NOC'] = df_events['NOC'].str.replace("IRI", "IRN")
df_events['NOC'] = df_events['NOC'].str.replace("GER", "DEU")
df_events['NOC'] = df_events['NOC'].str.replace("BAH", "BHS")
df_events['NOC'] = df_events['NOC'].str.replace("SUI", "CHE")
df_events['NOC'] = df_events['NOC'].str.replace("ISV", "VIR")
df_events['NOC'] = df_events['NOC'].str.replace("GRE", "GRC")
df_events['NOC'] = df_events['NOC'].str.replace("DEN", "DNK")
df_events['NOC'] = df_events['NOC'].str.replace("NED", "NLD")
df_events['NOC'] = df_events['NOC'].str.replace("CGO", "COG")
df_events['NOC'] = df_events['NOC'].str.replace("LAT", "LVA")
df_events['NOC'] = df_events['NOC'].str.replace("INA", "IDN")
df_events['NOC'] = df_events['NOC'].str.replace("GAM", "GMB")
df_events['NOC'] = df_events['NOC'].str.replace("GBS", "GNB")
df_events['NOC'] = df_events['NOC'].str.replace("MAS", "MYS")
df_events['NOC'] = df_events['NOC'].str.replace("NGR", "NGA")
df_events['NOC'] = df_events['NOC'].str.replace("VIN", "VCT")
df_events['NOC'] = df_events['NOC'].str.replace("BRU", "BRN")
df_events['NOC'] = df_events['NOC'].str.replace("SLO", "SVN")
df_events['NOC'] = df_events['NOC'].str.replace("MRI", "MUS")
df_events['NOC'] = df_events['NOC'].str.replace("BUL", "BGR")
df_events['NOC'] = df_events['NOC'].str.replace("PUR", "PRI")
df_events['NOC'] = df_events['NOC'].str.replace("MON", "MCO")
df_events['NOC'] = df_events['NOC'].str.replace("SKN", "KNA")
df_events['NOC'] = df_events['NOC'].str.replace("POR", "PRT")

# merge by stripped country names and make sure Olympic Data NOC matches WorldBank country_code
df_events = pd.merge(df_events, df_worldbanknames, how="left", left_on='Oclean', right_on = 'Wclean', suffixes=('', '_y'))
df_events['NOC'] = np.where(df_events['NOC_y'].isnull(), df_events['NOC'], df_events['NOC_y'])

# drop unncessary columns from the merge
df_events = df_events.drop(columns = ['Oclean', 'NOC_y', 'Wclean'], axis = 1)

In [8]:
#  step 3 - clean Country names based on the NOC (for small teams with Country NOCs)
df_events = pd.merge(df_events, df_worldbanknames, how="left", left_on='NOC', right_on = 'NOC', suffixes=('', '_y'))
df_events['Country'] = np.where(df_events['Country'].isnull(), df_events['Country_y'], df_events['Country'])
# drop unncessary columns from the merge
df_events = df_events.drop(columns = ['Country_y', 'Wclean'], axis = 1)

In [9]:
#  step 4 - clean Country names that do not match with World Bank, but have medals 
df_events_not_matching = pd.merge(df_events, df_worldbanknames, how = "left", left_on = ['NOC'], right_on = ['NOC'], suffixes=('', '_y'))
not_matching_NOC = df_events_not_matching[df_events_not_matching['Country_y'].isnull()]
not_matching_NOC = not_matching_NOC.groupby(['NOC','Team']).count().reset_index()
not_matching_NOC = not_matching_NOC.drop(columns = ['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight',
       'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'Country', 'Country_y', 'Wclean'], axis =1)

not_matching_NOC = not_matching_NOC[not_matching_NOC['Medal'] > 0]
max_medals_NOC = not_matching_NOC.groupby(['NOC'])['Medal'].transform(max) == not_matching_NOC['Medal']
max_medals_NOC = not_matching_NOC[max_medals_NOC]

df_events = pd.merge(df_events, max_medals_NOC, how="left", left_on='NOC', right_on = 'NOC', suffixes=('', '_y'))
df_events['Country'] = np.where(df_events['Country'].isnull(), df_events['Team_y'], df_events['Country'])
# drop unncessary columns from the merge
df_events = df_events.drop(columns = ['Team_y', 'Medal_y'], axis = 1)

In [10]:
# step 5 - use Team name when Country name is NaN
df_events['Country'] = np.where(df_events['Country'].isnull(), df_events['Team'], df_events['Country'])

In [11]:
# step 6 - filter Olympic games database to be only after 1960
df_events = df_events[df_events["Year"] >= 1960]

<h3> Define Classes for the needed sqlite tables

In [12]:
class Events(Base):
    __tablename__ = 'events'
    ID = Column(Integer, primary_key=True)
    Name = Column(String(255))
    Sex = Column(String(255))
    Age = Column(Integer)
    Height = Column(Integer)
    Weight = Column(Integer)
    Team = Column(String(255))
    NOC = Column(String(255))
    Games = Column(String(255))
    Year = Column(Integer, primary_key=True)
    Season = Column(String(255))
    City = Column(String(255))
    Sport = Column(String(255))
    Event = Column(String(255), primary_key=True)
    Medal = Column(String(255))
    Country = Column(String(255), primary_key=True)

In [13]:
class Events_Final(Base):
    __tablename__ = 'events_final'
    Year = Column(Integer, primary_key=True)
    Season = Column(String(255), primary_key=True)
    City = Column(String(255), primary_key=True)
    NOC = Column(String(255), primary_key=True)
    Country = Column(String(255), primary_key=True)
    Game_Label = Column(String(255))
    Chart_Label = Column(String(255))
    GDP = Column(Float)
    Population = Column(Integer)
    GDP_per_capita = Column(Float)
    No_olympians = Column(Integer)
    Bronze_athlete = Column(Integer)
    Gold_athlete = Column(Integer)
    Silver_athlete = Column(Integer)
    Total_Medals_athlete = Column(Integer)
    Bronze_team = Column(Integer)
    Gold_team = Column(Integer)
    Silver_team = Column(Integer)
    Total_Medals_team = Column(Integer)

<h3> Create SQLite database and connect to it

In [14]:
# Step 1 - Create engine connection, classes, session and connection
disk_engine = create_engine('sqlite:///olympic_events.sqlite')
Base.metadata.create_all(disk_engine)
session = Session(disk_engine)
conn = disk_engine.connect()

<h3> Upload events dataframe into SQLite database

In [15]:
# Step 2 - Export the df_events dataframe to sqlite
df_events.to_sql('events', disk_engine, if_exists='append', index = False)

<h3> Prepare the code for Events Final Table

In [16]:
# Base table
events_country = pd.read_sql_query('SELECT year, season, city, NOC, country FROM events \
GROUP BY year, season, city, NOC, country',disk_engine)
events_country['Game_Label'] = events_country['Season'] + ' ' + events_country['Year'].map(str) + ' - '  + events_country['City'] 
events_country['Chart_Label'] = events_country['Year'].map(str) + ' - '  + events_country['City'] 
events_country = events_country[['Year', 'Season', 'City', 'Game_Label', 'NOC', 'Chart_Label', 'Country']]

In [17]:
# Number of Athletes
# Step 1 Intermediate table
olympians_team_detail = pd.read_sql_query('SELECT year, season, name, NOC FROM events \
GROUP BY year, season, name, NOC',disk_engine)
olympians_team = olympians_team_detail.groupby(['Year','Season', 'NOC']).count()
olympians_team.reset_index(inplace = True)
olympians_team = olympians_team.rename(columns = {"Name": "No_olympians"})

In [18]:
# Number of Medals per athlete
# Step 1
medals_athlete_detail = pd.read_sql_query('SELECT year, season, NOC, sport, event, name, sex, medal FROM events \
GROUP BY year, season, NOC, sport, event, name, sex, medal',disk_engine)
# Step 2
medals_athlete = medals_athlete_detail.groupby(['Year','Season','NOC', 'Name','Medal']).count()[['Sport']]
medals_athlete.reset_index(inplace = True)
medals_athlete.rename(columns = {"Sport": "#Medals"}, inplace = True)
medals_athlete.sort_values(by=["#Medals"], ascending=False)
# Step 3
medals_athlete_pivot = medals_athlete.pivot_table(index=['Year','Season','NOC','Name'], values = '#Medals', columns='Medal', aggfunc='sum')
medals_athlete_pivot.reset_index(inplace = True)
medals_athlete_pivot.replace(np.nan,0, inplace = True)

# Step 4
medals_athlete_pivot['Gold_athlete'] = np.where(medals_athlete_pivot['Gold'] > 0, 1, 0)
medals_athlete_pivot['Silver_athlete'] = np.where((medals_athlete_pivot['Gold_athlete'] == 0) & (medals_athlete_pivot['Silver'] > 0), 1, 0)
medals_athlete_pivot['Bronze_athlete'] = np.where((medals_athlete_pivot['Gold_athlete'] == 0) & (medals_athlete_pivot['Silver_athlete'] == 0), 1, 0)
medals_athlete_pivot["Total_Medals_athlete"] = medals_athlete_pivot["Bronze_athlete"] + medals_athlete_pivot["Gold_athlete"] + medals_athlete_pivot["Silver_athlete"]

medals_athlete_total = medals_athlete_pivot.groupby(['Year','Season','NOC']).sum()[['Silver_athlete','Bronze_athlete','Gold_athlete','Total_Medals_athlete']]
medals_athlete_total.reset_index(inplace = True)

In [19]:
# Number of Medals per sport
# Step 1
medals_sport_detail = pd.read_sql_query('SELECT year, season, NOC, sport, event, sex, medal FROM events \
GROUP BY year, season, NOC, sport, event, sex, medal',disk_engine)
# Step 2
medals_sport = medals_sport_detail.groupby(['Year','Season','NOC', 'Medal']).count()[['Event']]
medals_sport.reset_index(inplace = True)
medals_sport.rename(columns = {"Event": "#Medals"}, inplace = True)
medals_sport.sort_values(by=["#Medals"], ascending=False)
# Step 3
medals_sport_total = medals_sport.pivot_table(index=['Year','Season','NOC'], values = '#Medals', columns='Medal', aggfunc='sum')
medals_sport_total.reset_index(inplace = True)
medals_sport_total.replace(np.nan,0, inplace = True)
medals_sport_total = medals_sport_total.rename(columns = {"Bronze": "Bronze_team", "Gold":"Gold_team", "Silver": "Silver_team"})
medals_sport_total["Total_Medals_team"] = medals_sport_total["Bronze_team"] + medals_sport_total["Gold_team"] + medals_sport_total["Silver_team"]

In [20]:
# Final data gathering to reach Events_final

# step 1 - merging tables
events_merge_1 = pd.merge(events_country, df_pop_gdp_final, how = "left", left_on = ['Year','NOC'], right_on = ['Year','NOC'], suffixes=('', '_y'))
events_merge_1 = events_merge_1.drop(columns = ['Country_y'])
events_merge_2 = pd.merge(events_merge_1, olympians_team, how = "left", left_on = ['Year','Season', 'NOC'], right_on = ['Year','Season', 'NOC'])
events_merge_3 = pd.merge(events_merge_2, medals_athlete_total, how = "left", left_on = ['Year','Season', 'NOC'], right_on = ['Year','Season', 'NOC'])
events_merge_4 = pd.merge(events_merge_3, medals_sport_total, how = "left", left_on = ['Year','Season', 'NOC'], right_on = ['Year','Season', 'NOC']) 

In [21]:
# step 2 - prepare aggregated value for all countries - World (WLD)
wld_events_1 = events_merge_4.groupby(['Year','Season','City','Game_Label', 'Chart_Label']).sum()
wld_events_2 = pd.DataFrame(wld_events_1).reset_index()
wld_events_2['NOC'] = 'WLD'
wld_events_2['Country'] = 'World'

# add the WLD Pop and GDP information
wld_base_info = df_pop_gdp_final[df_pop_gdp_final['NOC'] == 'WLD']
wld_all_info = pd.merge(wld_events_2, wld_base_info, how = "left", left_on = ['NOC','Year','Country'], right_on = ['NOC','Year','Country'], suffixes=('_x', ''))
wld_all_info = wld_all_info.drop(columns = ['GDP_x', 'Population_x', 'GDP_per_capita_x'], axis = 1)

In [22]:
# step 3 - add aggregated value for all countries - World (WLD) in Events_Final
events_pre_final = pd.concat([events_merge_4, wld_all_info], sort=False)

In [23]:
# step 4 - drop countries not recognized by World Bank and without medals over the years
merge_wld_bank = pd.merge(events_pre_final, df_worldbanknames, how = "left", left_on = ['NOC'], right_on = ['NOC'], suffixes=('', '_y'))

# NOCs not matching World Bank
not_matching_NOC = merge_wld_bank[merge_wld_bank['Country_y'].isnull()]
not_matching_NOC = not_matching_NOC.groupby(['NOC','Country']).sum().reset_index()
not_matching_NOC = not_matching_NOC.drop(columns = ['Year', 'GDP', 'Population', 'GDP_per_capita',
       'No_olympians', 'Bronze_athlete', 'Gold_athlete', 'Silver_athlete',
       'Total_Medals_athlete', 'Bronze_team', 'Gold_team', 'Silver_team'], axis =1)

# drop rows with columns not maching world bank and without medals
not_matching_NOC_no_medals = not_matching_NOC[not_matching_NOC['Total_Medals_team'] == 0]
not_matching_NOC_no_medals = not_matching_NOC_no_medals.groupby('NOC').all().reset_index()
not_matching_NOC_no_medals = not_matching_NOC_no_medals.drop(columns = ['Total_Medals_team'])

events_pre_final = pd.merge(events_pre_final, not_matching_NOC_no_medals, how = "left", left_on = ['NOC'], right_on = ['NOC'], suffixes=('', '_Drop'))
indexNames = events_pre_final[events_pre_final['Country_Drop'] == True].index
events_pre_final.drop(indexNames , inplace=True)
events_pre_final = events_pre_final.drop(columns = ['Country_Drop'], axis = 1)

In [24]:
# step 5 - create events_final
events_final = events_pre_final

<h3> Load Events Final Table into SQLite

In [25]:
# step 6 - load the events_final in SQLite
events_final.to_sql('events_final', disk_engine, if_exists='append', index = False)

### Load Events Final Table into JSON file

In [ ]:
# step 7 - load the events_final into JSON file for choropleth map
events_final.to_json('medal_counts.json')